In [1]:
#将四氢异喹啉生物碱的文献doi和摘要的excel文件加上'custom_id‘
import pandas as pd
import os

# 读取Excel文件
input_path = r'/kaggle/input/test-data/test100.xlsx'
df = pd.read_excel(input_path)

# 检查是否已存在custom_id列，如果不存在则添加
if 'custom_id' not in df.columns:
    # 为每一行生成自定义格式的ID (R00001, R00002, ...)
    df['custom_id'] = [f"R{str(i+1).zfill(5)}" for i in range(len(df))]
    
    # 保存修改后的Excel文件到可写目录
    output_path = r'/kaggle/working/test100_with_custom_id.xlsx'
    df.to_excel(output_path, index=False)
    
    print(f"已成功添加custom_id列，文件保存为: {output_path}")
    print(f"共处理了 {len(df)} 行数据")
    print(f"ID范围: {df['custom_id'].iloc[0]} 到 {df['custom_id'].iloc[-1]}")
    
    
output_path = r'/kaggle/working/test100_with_custom_id.xlsx'
df.to_excel(output_path, index=False)
print(f"已将现有数据保存到: {output_path}")

已成功添加custom_id列，文件保存为: /kaggle/working/test100_with_custom_id.xlsx
共处理了 99 行数据
ID范围: R00001 到 R00099
已将现有数据保存到: /kaggle/working/test100_with_custom_id.xlsx


In [2]:
#生成批量推理的jsonl数据集
import json

import pandas as pd
from tqdm import tqdm


datas=pd.read_excel(r'/kaggle/working/test100_with_custom_id.xlsx')

def generate_prompt(abstract):
    return f"""
    请严格遵循以下规则分析这篇摘要：
（注意：回答必须使用JSON格式，严格按照下面的输出格式输出，无额外解释。若文献未提及相关内容，字段值应为 `null`）

# 判断逻辑
1. ​**四氢异喹啉类生物碱判定**​（需同时满足）：
   - 明确提及化合物属于"四氢异喹啉类生物碱"（tetrahydroisoquinoline alkaloids）或具体亚类（如苄基异喹啉类）
   - 排除以下干扰项：
     - 单纯提及四氢异喹啉类（无生物碱结构）
     - 非四氢异喹啉类生物碱
     - 模糊表述（如"可能含有生物碱"）

2. ​**研究方向判定**​（若`has_tetrahydroisoquinoline_alkaloids`为true）：
   - ​**结构鉴定**​：需包含"分离(isolation)"和"结构鉴定(structure elucidation)"等证据
   - ​**活性研究**​：需提供具体靶点名称及实验数据（如IC50）
   - ​**生物合成**​：需明确前体、产物、关键酶的三要素

# 输出规则
1. 字段层级关系：
   - `compounds`数组中每个对象必须包含`classification`字段，取值示例：
     - benzylisoquinoline alkaloids（苄基异喹啉生物碱）
     - 1-benzylisoquinolines alkaloids（1-苄基异喹啉）
     - bisbenzylisoquinolines alkaloids（双苄基异喹啉）
     - morphinans alkaloids（吗啡烷）
     - aporphines alkaloids（阿朴啡）
     - protoberberines alkaloids（原小檗碱）
     - phthalideisoquinolines alkaloids（苯并菲啶）
     - benzophenanthridines alkaloids（苯酞异喹啉）
     - protopines alkaloids（原阿片）
     - pavines alkaloids
     - phenethylisoquinoline alkaloids（苯乙基异喹啉生物碱）
     - Amaryllidaceae alkaloids（石蒜科生物碱）
     - ipecac alkaloids （吐根生物碱）或 emetine（吐根碱）
   - 每个化合物的生物活性(`bioactivity`)和物种来源(`species`)独立关联
   - 未被文献明确提及的字段必须设为 `null`

2. 数据提取规范：
   - 分类名称：必须使用英文术语（如benzylisoquinoline alkaloids）
   - 化合物名称：优先使用文献中的命名（如Plumulane A）
   - 物种拉丁名：必须使用斜体格式（如 _"Fissistigma polyanthum"_）
   - 靶点名称：精确到分子级别（如AChE 而非"抗氧化活性"）

# 输出指定JSON格式结果（严格按照这个格式输出,不需要输出"reasoning_content"）
{{
  "has_tetrahydroisoquinoline_alkaloids": true,
  "research_type": ["structure", "activity", "biosynthesis"],
  "compounds": [
    {{
      "name": "Plumulane A",
      "classification": "benzylisoquinoline alkaloids",
      "species": {{
        "source": ["_Fissistigma polyanthum_"], 
        "part_used": "roots"
      }},
      "bioactivity": [
        {{
          "target": "EGFR kinase",
          "mechanism": "competitive inhibition",
          "value": "IC50 = 2.3 μM",
          "cell_line": "HepG2"
        }}
      ]
    }}
  ],
  "biosynthesis": {{
    "routes": [
      {{
        "substrate": "geranylgeranyl diphosphate",
        "product": "pendulumine A",
        "key_enzymes": ["CYP719A21"],
        "evidence": ["isotope labeling"]
      }}
    ]
  }}
}}

# 待分析摘要：
{abstract}
"""

output_file = "./input_abstracts.jsonl"
with open(output_file, 'w', encoding='utf-8') as f:
    for i in tqdm(range(len(datas))):
        abstract = datas.loc[i, 'Abstract']
        custom_id = datas.loc[i, 'custom_id']
        jsonl_obj = {
            "custom_id": f"{custom_id}",
            "body": {
                "messages": [
                    {
                        "role": "system",
                        "content": "您作为四氢异喹啉类生物碱的研究专家，您需要根据文献的摘要，锁定关键信息，并输出指定JSON格式结果。"
                    },
                    {
                        "role": "user",
                        "content": generate_prompt(abstract)
                    }
                ],
                "temperature": 0.0
            }
        }
        f.write(json.dumps(jsonl_obj, ensure_ascii=False) + '\n')

100%|██████████| 99/99 [00:00<00:00, 10337.23it/s]


In [3]:
#检查生成的数据集是否有格式错误
import json

def check_jsonl_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        total = 0
        custom_id_set = set()
        for line in file:
            if line.strip() == "":
                continue
            try:
                line_dict = json.loads(line)
            except json.decoder.JSONDecodeError:
                raise Exception(f"批量推理输入文件格式错误，第{total + 1}行非json数据")
            if not line_dict.get("custom_id"):
                raise Exception(f"批量推理输入文件格式错误，第{total + 1}行custom_id不存在")
            if not isinstance(line_dict.get("custom_id"), str):
                raise Exception(f"批量推理输入文件格式错误, 第{total + 1}行custom_id不是string")
            if line_dict.get("custom_id") in custom_id_set:
                raise Exception(
                    f"批量推理输入文件格式错误，custom_id={line_dict.get('custom_id', '')}存在重复"
                )
            else:
                custom_id_set.add(line_dict.get("custom_id"))
            if not isinstance(line_dict.get("body", ""), dict):
                raise Exception(
                    f"批量推理输入文件格式错误，custom_id={line_dict.get('custom_id', '')}的body非json字符串"
                )
            total += 1
    return total


# 替换<YOUR_JSONL_FILE>为你的JSONL文件路径
file_path = "/kaggle/working/input_abstracts.jsonl"
total_lines = check_jsonl_file(file_path)
print(f"文件中有效JSON数据的行数为: {total_lines}")


文件中有效JSON数据的行数为: 99
